# SemEval-2020 Task 4: Commonsense Validation and Explanation

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1rlwPIErsvGcYYgM6s_FT9FDn9lcHzvVU?usp=sharing)

# Getting started

In [1]:
!nvidia-smi

Sat Jun  5 11:19:49 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Mount onto google drive 

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Position into correct folder

In [7]:
%cd /content/gdrive/MyDrive/git/fer-tar

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar


# Install packages

In [8]:
!pip install -q pytorch_lightning
!pip install -q transformers

In [5]:
from importlib import reload

In [9]:
from src.modelss import TransformerBasedClassifier
from src.tokenizers import TransformerTokenizer
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint
from my_scripts.data import ComVEDataset
from torch.utils.data import DataLoader
from itertools import product


In [ ]:
import src.modelss
import my_scripts.data
reload(src.modelss)
reload(my_scripts.data)

<module 'my_scripts.data' from '/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar/my_scripts/data.py'>

# Constants

In [10]:
BATCH_SIZE = 32
EPOCHS = 8
NAMES = ['electra']

# Testing...

In [14]:
%cd Marko

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar/Marko


In [26]:
for name in NAMES:
    for drop in (0.8,):
        train_ds, val_ds, test_ds = ComVEDataset.gpt2_lm_plain(data_folder='/content/gdrive/MyDrive/git/data',
                                                                  training_x='backtranslation_data3.csv',
                                                                  training_y='backtranslation_labels.csv',
                                                                  x_transforms=[TransformerTokenizer(name=name)])
        train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
        val_dataloader = DataLoader(val_ds, batch_size=len(val_ds))
        test_dataloader = DataLoader(test_ds, batch_size=len(test_ds))
        
        training_steps = len(train_dataloader) * EPOCHS
        print(training_steps)
        model = TransformerBasedClassifier(name=name, drop=0.1, lr=4e-5, weight_decay=0.01, training_steps=training_steps)
        checkpoint_callback = ModelCheckpoint(
            monitor = 'val_loss',
            dirpath = 'checkpoint_gpt2_lm_plain/',
            filename = 'best-model-' + name,
            save_top_k = 1
        )

        trainer = pl.Trainer(max_epochs=EPOCHS, gpus=1, progress_bar_refresh_rate=20, callbacks=[checkpoint_callback])
        trainer.fit(model, train_dataloader, val_dataloader)
        l, a = model.get_best_loss_and_acc()
        model = TransformerBasedClassifier.load_from_checkpoint(checkpoint_callback.best_model_path,
                                                                name=name, drop=drop, append_name='large-cemb')
        model.val_loss, model.val_acc = l, a

        trainer.test(model, test_dataloader)
 
    !rm -rf checkpoint_gpt2_lm_plain/

4536


Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name        | Type         | Params
---------------------------------------------
0 | transformer | ElectraModel |

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.9470000267028809, 'test_loss': 0.14849627017974854}
--------------------------------------------------------------------------------


In [ ]:
import src.modelss
import my_scripts.data
reload(src.modelss)
reload(my_scripts.data)

<module 'my_scripts.data' from '/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar/my_scripts/data.py'>

In [ ]:
a = [np.array(2), np.array(3)]
print(np.array(a))

[2 3]


In [ ]:
import numpy as np

In [ ]:
max([1, 2, 3])

3

In [ ]:
from statsmodels.stats.contingency_tables import mcnemar


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import json
a = None
b = None
c = None
with open('electra-gpt2_plain1.json') as f:
    a = json.load(f)
    a = a['predictions']

with open('electra-gpt2_plain2.json') as f:
    c = json.load(f)
    c = c['predictions']

with open('/content/gdrive/MyDrive/git/fer-tar/results/gpt2_lm_plain_log/electra-gpt2_lm_plain_log.json') as f:
    b = json.load(f)
    b = b['predictions']

In [ ]:
from sklearn.metrics import confusion_matrix as cm

In [ ]:
import numpy as np

In [19]:
!pwd

/content/gdrive/.shortcut-targets-by-id/1yfuNPQUT_G0CfNtdGxHhLi96giKyswbZ/git/fer-tar/Marko


In [ ]:
a = np.array(a)
b = np.array(b)
c = np.array(c)
# a = a >= 0.5
# b = b >= 0.5
# c = c >= 0.5

In [ ]:
from scipy.stats import ttest_rel, f_oneway


In [ ]:
ttest_rel(a, c)

Ttest_relResult(statistic=-0.9004634332088931, pvalue=0.3680906909231231)

In [ ]:
f_oneway(a, b, c)

F_onewayResult(statistic=0.11883698929716709, pvalue=0.8879567186613047)

In [ ]:
d = np.abs(a - b)
e = np.abs(a - c)
f = np.abs(b - c)

In [ ]:
f.mean()

0.10387365626955489

In [ ]:
ttest_rel(d, np.zeros(a.shape))

Ttest_relResult(statistic=19.285942614016946, pvalue=1.0597595839361604e-70)

In [ ]:
print(mcnemar(cm(b, c)))

pvalue      0.16057841913397983
statistic   36.0
